In [ ]:
!pip install jax jaxlib
!pip install --quiet --upgrade scipy
!pip install --quiet jax jaxlib optax

In [ ]:
import jax
from jax.scipy.stats import norm
import jax.numpy as jnp
from scipy.stats import norm
import time
import jax.numpy as jnp
from numpy.polynomial.legendre import leggauss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar, brentq, minimize
from scipy.special import gamma
from numpy.polynomial.legendre import leggauss
import warnings
warnings.filterwarnings('ignore')
from numpy.random import default_rng
from math import log
from numpy.random import default_rng, SeedSequence
from scipy.stats import kstwobign, cramervonmises, uniform
from joblib import Parallel, delayed
from itertools import zip_longest
from collections import OrderedDict


# ==============================================================
# Risk Measure
# ==============================================================


In [ ]:


# 1) Load
path = "/content/sample_data/norwegianfire_raw.csv"
df = pd.read_csv(path)
df["year_full"] = 1900 + df["year"].astype(int)

# -------- Choose the cap in NOK -----------------

MONSTER_NOK = 2_000_000_000  # 2 billion NOK   (alternative)

MONSTER_kNOK = MONSTER_NOK / 1_000.0

# 2) Strict threshold: keep claims EXCEEDING 500 kNOK
theta_kNOK = 500.0
df_work = df.loc[df["size"] > theta_kNOK].copy()
n_total = len(df)
n_work  = len(df_work)

# 3) Helper for "nearest" empirical quantile (robust across pandas/numpy versions)
def q_nearest(x: np.ndarray, p: float) -> float:
    x = np.asarray(x, float)
    x = x[~np.isnan(x)]
    if x.size == 0:
        return np.nan
    s = pd.Series(x)
    try:
        return float(s.quantile(p, method="nearest"))
    except TypeError:
        try:
            return float(s.quantile(p, interpolation="nearest"))
        except TypeError:
            try:
                return float(np.quantile(x, p, method="nearest"))
            except TypeError:
                return float(np.percentile(x, 100*p, interpolation="nearest"))

# 4) Headline stats (kNOK)
s = df_work["size"].astype(float)
n = s.size
m1 = s.mean()
q1 = q_nearest(s.values, 0.25)
q3 = q_nearest(s.values, 0.75)

# Unbiased (Fisher–Pearson) sample skewness
m2 = np.mean((s - m1)**2)
m3 = np.mean((s - m1)**3)
g1 = m3 / (m2**1.5) if m2 > 0 else np.nan
skew_unbiased = (np.sqrt(n*(n-1))/(n-2))*g1 if n > 2 else np.nan

print("=== Norwegian fire insurance (kNOK), strict > 500 ===")
print(f"n_total: {n_total}")
print(f"n_after_theta (x>500): {n_work}")
print(f"year_min_max: ({int(df['year_full'].min())}, {int(df['year_full'].max())})")
print(f"min_kNOK: {s.min():,.0f}")
print(f"max_kNOK: {s.max():,.0f}")
print(f"mean_kNOK: {m1:,.3f}")
print(f"q1_kNOK: {q1:,.0f}")
print(f"q3_kNOK: {q3:,.0f}")
print(f"skewness: {skew_unbiased:.2f}")

# ---------- Build the four arrays (all in kNOK) ----------
# 1) Original data (x > 500)
x_original = np.asarray(df_work["size"].values, dtype=float)
x_original = np.sort(x_original)

# 2) Modified original: replace the single maximum with MONSTER_kNOK
x_mod_original = x_original.copy()
imax = np.argmax(x_mod_original)
x_mod_original[imax] = MONSTER_kNOK
x_mod_original = np.sort(x_mod_original)

# 3) Sampled: n=50, seed=123
rng = np.random.default_rng(123)
idx50 = rng.choice(x_original.shape[0], size=50, replace=False)
x_sampled = np.sort(x_original[idx50])

# 4) Modified sampled: replace its maximum with MONSTER_kNOK
x_mod_sampled = x_sampled.copy()
jmax = np.argmax(x_mod_sampled)
x_mod_sampled[jmax] = MONSTER_kNOK
x_mod_sampled = np.sort(x_mod_sampled)

# 5) Wire up θ for the ETLL code that uses Greek theta
θ = theta_kNOK

# Quick sanity prints
print(f"\nCounts: {len(x_original)}, {len(x_mod_original)}, {len(x_sampled)}, {len(x_mod_sampled)}")
print(f"Original max -> Modified original max: {np.max(x_original):,.0f} -> {np.max(x_mod_original):,.0f}")
print(f"Sampled max  -> Modified sampled max : {np.max(x_sampled):,.0f}  -> {np.max(x_mod_sampled):,.0f}")
print(f"θ (fixed): {θ:,.0f} kNOK")
print(f"Monster (cap): {MONSTER_kNOK:,.0f} kNOK  = {int(MONSTER_NOK):,} NOK")


=== Norwegian fire insurance (kNOK), strict > 500 ===
n_total: 9181
n_after_theta (x>500): 9020
year_min_max: (1972, 1992)
min_kNOK: 501
max_kNOK: 465,365
mean_kNOK: 2,247.860
q1_kNOK: 711
q3_kNOK: 1,817
skewness: 30.61

Counts: 9020, 9020, 50, 50
Original max -> Modified original max: 465,365 -> 2,000,000
Sampled max  -> Modified sampled max : 4,607  -> 2,000,000
θ (fixed): 500 kNOK
Monster (cap): 2,000,000 kNOK  = 2,000,000,000 NOK


In [ ]:
# ==== Print the sampled values in a 5×10 grid ======

def print_sample_block(x, title, rows=5, cols=10, integers=True):
    """
    Print values as a rows×cols block, left-to-right then top-to-bottom,
    with thousands separators. Assumes len(x) == rows*cols (e.g., 50).
    """
    x = np.sort(np.asarray(x, float))
    if x.size != rows*cols:
        raise ValueError(f"Expected {rows*cols} values, got {x.size}. "
                         f"Adjust rows/cols or sample size.")
    if integers:
        x_disp = np.round(x).astype(int)
        fmt = lambda v: f"{v:,.0f}"
    else:
        x_disp = x
        fmt = lambda v: f"{v:,.3f}"

    print(title)
    for r in range(rows):
        start = r*cols
        row_vals = [fmt(v) for v in x_disp[start:start+cols]]
        print(", ".join(row_vals))
    print()  # blank line after the block

# ---- Print both blocks (kNOK) ----
print_sample_block(x_sampled,      "Sampled data (n=50, kNOK):")
print_sample_block(x_mod_sampled,  "Modified sampled data (n=50, kNOK):")


Sampled data (n=50, kNOK):
505, 512, 513, 520, 541, 615, 632, 641, 650, 650
675, 675, 682, 698, 699, 700, 704, 704, 718, 735
745, 805, 822, 901, 911, 942, 957, 961, 1,035, 1,060
1,075, 1,223, 1,244, 1,343, 1,500, 1,743, 1,750, 2,039, 2,072, 2,097
2,139, 2,220, 2,294, 2,348, 2,468, 2,764, 3,317, 3,814, 4,377, 4,607

Modified sampled data (n=50, kNOK):
505, 512, 513, 520, 541, 615, 632, 641, 650, 650
675, 675, 682, 698, 699, 700, 704, 704, 718, 735
745, 805, 822, 901, 911, 942, 957, 961, 1,035, 1,060
1,075, 1,223, 1,244, 1,343, 1,500, 1,743, 1,750, 2,039, 2,072, 2,097
2,139, 2,220, 2,294, 2,348, 2,468, 2,764, 3,317, 3,814, 4,377, 2,000,000



In [ ]:

# ---- optional: keep BLAS threads = 1 per worker to avoid oversubscription
try:
    from threadpoolctl import threadpool_limits
    _HAS_TPCTL = True
except Exception:
    _HAS_TPCTL = False

# ---------------- Performance knobs ----------------
N_JOBS       = -1       # use all cores
BACKEND      = "loky"   # processes for CPU-bound work
BLAS_THREADS = 1        # 1 BLAS thread per worker

# Numerical effort: high for observed fit, lighter for MC refits
NQUAD_OBS = 800
NQUAD_MC  = 200
GRID_OBS  = 401
GRID_MC   = 201
MC_BATCH  = 200         # chunk MC jobs to lower overhead

# ---- Monte-Carlo reps
# Use big B for full-sample fixed-parameter MC
MC_B_ORIG      = 10000
MC_B_MOD_ORIG  = 10000
# Moderate B for n=50 refitting MC
MC_B_SAMP      = 2000
MC_B_MOD_SAMP  = 2000

ln2 = np.log(2.0)

# ---------- Core ETLL pieces (θ fixed) ----------
def etll_cdf(x, alpha, beta, theta):
    x = np.asarray(x, float)
    u = np.zeros_like(x, dtype=float)
    m = x > theta
    if not np.any(m):
        return u
    t = (theta / x[m])**alpha
    if abs(beta) < 1e-8:
        u[m] = 1.0 - (np.log1p(t) / ln2)
    else:
        two_b = np.exp(beta * ln2)
        u[m] = (two_b - np.power(1.0 + t, beta)) / (two_b - 1.0)
    return np.clip(u, 0.0, 1.0)

def ks_statistic(x, cdf):
    x = np.sort(np.asarray(x, float)); n = x.size
    u = cdf(x); ecdf = (np.arange(1, n+1)) / n
    return float(np.max(np.abs(u - ecdf)))

def cvm_statistic(x, cdf):
    x = np.sort(np.asarray(x, float)); n = x.size
    u = cdf(x); i = np.arange(1, n+1)
    return float(np.sum((u - (2*i - 1)/(2*n))**2) + 1.0/(12*n))

def etll_sample(n, alpha, beta, theta, rng=None):
    rng = default_rng() if rng is None else rng
    u = rng.uniform(0.0, 1.0, int(n))
    if abs(beta) < 1e-8:
        t = np.expm1((1.0 - u) * ln2)
    else:
        two_b = np.exp(beta * ln2)
        base  = two_b - (two_b - 1.0) * u
        base  = np.maximum(base, 1e-300)
        t     = np.expm1(np.log(base)/beta)
    return theta * np.power(t, -1.0/alpha)

# ---------- MLE (θ fixed) ----------
def etll_loglik_alpha_beta(x, alpha, beta, theta):
    if alpha <= 0 or abs(beta) > 12:
        return -np.inf
    xv = np.asarray(x, float); xv = xv[xv > theta]
    if xv.size == 0:
        return -np.inf
    n = xv.size
    two_b = np.exp(beta * ln2)
    denom = two_b - 1.0
    if abs(denom) < 1e-14:
        denom = beta * ln2 + 0.5*(beta**2)*(ln2**2)
    const = np.log(alpha) + np.log(abs(beta)) - np.log(abs(denom))
    ratio = (theta / xv)**alpha
    ll = n*const - np.sum(np.log(xv)) + np.sum(alpha*np.log(theta/xv)) + (beta-1.0)*np.sum(np.log1p(ratio))
    return float(ll)

def fit_mle_etll(x, theta):
    x = np.asarray(x, float); x = x[x > theta]
    if x.size < 5:
        return np.nan, np.nan
    lx = np.log(x); m2 = np.mean((lx - lx.mean())**2)
    a0 = max(0.1, 1.0/np.sqrt(max(m2, 1e-6))); b0 = 0.5
    def nll(p):
        a,b = p; v = etll_loglik_alpha_beta(x, a, b, theta)
        return -v if np.isfinite(v) else 1e20
    res = minimize(nll, x0=[a0, b0], bounds=[(1e-3, 40.0), (-12.0, 12.0)], method="L-BFGS-B")
    if res.success:
        return float(res.x[0]), float(res.x[1])
    res = minimize(nll, x0=[1.5, 0.2], bounds=[(1e-3, 40.0), (-12.0, 12.0)], method="L-BFGS-B")
    return (float(res.x[0]), float(res.x[1])) if res.success else (np.nan, np.nan)

# ---------- Robust L-estimation J(a,b) (θ fixed) ----------
def _ck_tau(beta, a, b, n_quad):
    nodes, w = np.polynomial.legendre.leggauss(n_quad)
    u = 0.5*(nodes + 1.0); w = 0.5*w
    J = a*b * (u**(a-1.0)) * ((1.0 - u**a)**(b-1.0))

    if abs(beta) < 1e-10:
        t = np.expm1((1.0 - u) * ln2)
    else:
        two_b = np.exp(beta * ln2)
        base  = two_b - (two_b - 1.0) * u
        base  = np.maximum(base, 1e-300)
        t     = np.expm1(np.log(base)/beta)

    t   = np.maximum(t, 1e-300)
    ell = np.log(t)

    c1  = float(np.sum(w * J * ell))
    c2  = float(np.sum(w * J * (ell**2)))
    tau = c2 - c1**2
    return c1, max(tau, 1e-14)

def fit_L_etll_stable(x, a=1.1, b=1.2, theta=500.0, n_quad=NQUAD_OBS, root_grid=GRID_OBS):
    x = np.asarray(x, float); x = x[x > theta]
    n = x.size
    if n < 5:
        return np.nan, np.nan

    xs  = np.sort(x)
    i   = np.arange(1, n+1)
    uo  = i/(n+1.0)

    Jw  = a*b * (uo**(a-1.0)) * ((1.0 - uo**a)**(b-1.0))
    wts = Jw / np.sum(Jw)

    lx = np.log(xs)
    mu1   = float(np.sum(wts * lx))
    mu2   = float(np.sum(wts * (lx**2)))
    Delta = max(mu2 - mu1**2, 1e-12)

    target = (np.log(theta) - mu1) / np.sqrt(Delta)

    def R(beta):
        c1, tau = _ck_tau(beta, a, b, n_quad=n_quad)
        return c1/np.sqrt(tau) - target

    grid = np.linspace(-10.0, 10.0, int(root_grid))
    vals = np.array([R(bi) for bi in grid])
    sgn  = np.sign(vals)

    beta_hat = None
    for k in range(len(grid)-1):
        if np.isfinite(vals[k]) and np.isfinite(vals[k+1]) and sgn[k]*sgn[k+1] < 0:
            beta_hat = brentq(lambda z: R(z), grid[k], grid[k+1], xtol=1e-8, maxiter=400)
            break

    if beta_hat is None:
        from scipy.optimize import minimize_scalar
        obj = lambda b: (R(b) if np.isfinite(R(b)) else 1e6)**2
        res = minimize_scalar(obj, bounds=(-10.0, 10.0), method="bounded",
                              options={"xatol":1e-8, "maxiter":1000})
        beta_hat = float(res.x)

    _, tau_hat = _ck_tau(beta_hat, a, b, n_quad=n_quad)
    alpha_hat  = np.sqrt(tau_hat / Delta)
    return float(alpha_hat), float(beta_hat)

def make_safe_L_factory(a_req, b_req, fallbacks=None, n_quad=NQUAD_OBS, root_grid=GRID_OBS):
    if fallbacks is None:
        fallbacks = [(1.05,1.10), (1.10,1.20), (1.20,1.30)]
    def fit_fun(z, _nq=n_quad, _rg=root_grid):
        ah, bh = fit_L_etll_stable(z, a_req, b_req, θ, n_quad=_nq, root_grid=_rg)
        if np.isfinite(ah) and np.isfinite(bh):
            return ah, bh
        for (aa,bb) in fallbacks:
            ah2, bh2 = fit_L_etll_stable(z, aa, bb, θ, n_quad=_nq, root_grid=_rg)
            if np.isfinite(ah2) and np.isfinite(bh2):
                return ah2, bh2
        return fit_mle_etll(z, θ)
    return fit_fun

# ---- MC p-values with RE-FITTING (n=50 panels)
def mc_pvals_parallel(x, fit_fun, theta, B=2000, seed=1234, n_jobs=N_JOBS, backend=BACKEND,
                      nquad_obs=NQUAD_OBS, nquad_mc=NQUAD_MC, grid_obs=GRID_OBS, grid_mc=GRID_MC,
                      batch=MC_BATCH):
    if _HAS_TPCTL: threadpool_limits(BLAS_THREADS)
    x = np.asarray(x, float); x = x[x > theta]
    n = x.size
    if n == 0: return (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)

    def fit_obs(z): return fit_fun(z, _nq=nquad_obs, _rg=grid_obs)
    a_hat, b_hat = fit_obs(x)
    cdf_hat = lambda z: etll_cdf(z, a_hat, b_hat, theta)
    D_obs = ks_statistic(x, cdf_hat); W_obs = cvm_statistic(x, cdf_hat)

    seeds = SeedSequence(seed).spawn(B)
    seed_ints = [int(s.generate_state(1)[0]) for s in seeds]

    def one_rep(seed_i):
        rng = default_rng(seed_i)
        xb = etll_sample(n, a_hat, b_hat, theta, rng)
        ah, bh = fit_fun(xb, _nq=nquad_mc, _rg=grid_mc)
        cdf_b = lambda z, aa=ah, bb=bh: etll_cdf(z, aa, bb, theta)
        return ks_statistic(xb, cdf_b), cvm_statistic(xb, cdf_b)

    def _chunks(lst, k):
        for i in range(0, len(lst), k): yield lst[i:i+k]

    Ds_all, Ws_all = [], []
    for chunk in _chunks(seed_ints, MC_BATCH):
        Ds, Ws = zip(*Parallel(n_jobs=n_jobs, backend=backend)(
            delayed(one_rep)(si) for si in chunk
        ))
        Ds_all.append(np.array(Ds)); Ws_all.append(np.array(Ws))
    Ds = np.concatenate(Ds_all); Ws = np.concatenate(Ws_all)

    # Davison–Hinkley
    p_ks  = (1.0 + np.sum(Ds >= D_obs)) / (B + 1.0)
    p_cvm = (1.0 + np.sum(Ws >= W_obs)) / (B + 1.0)
    return (a_hat, b_hat, D_obs, W_obs, p_ks, p_cvm)

# ---- MC p-values with FIXED PARAMETERS
def mc_pvals_fixedparams(x, fit_fun, theta, B=10000, seed=1234, n_jobs=N_JOBS, backend=BACKEND,
                         nquad_obs=NQUAD_OBS, batch=MC_BATCH):
    if _HAS_TPCTL: threadpool_limits(BLAS_THREADS)
    x = np.asarray(x, float); x = x[x > theta]
    n = x.size
    if n == 0: return (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)

    # Fit once (full accuracy), then DO NOT refit in replicates
    a_hat, b_hat = fit_fun(x, _nq=nquad_obs, _rg=GRID_OBS)
    cdf_hat = lambda z: etll_cdf(z, a_hat, b_hat, theta)
    D_obs = ks_statistic(x, cdf_hat); W_obs = cvm_statistic(x, cdf_hat)

    seeds = SeedSequence(seed).spawn(B)
    seed_ints = [int(s.generate_state(1)[0]) for s in seeds]

    def one_rep(seed_i):
        rng = default_rng(seed_i)
        xb = etll_sample(n, a_hat, b_hat, theta, rng)
        # NO refit here:
        return ks_statistic(xb, cdf_hat), cvm_statistic(xb, cdf_hat)

    def _chunks(lst, k):
        for i in range(0, len(lst), k): yield lst[i:i+k]

    Ds_all, Ws_all = [], []
    for chunk in _chunks(seed_ints, batch):
        Ds, Ws = zip(*Parallel(n_jobs=n_jobs, backend=backend)(
            delayed(one_rep)(si) for si in chunk
        ))
        Ds_all.append(np.array(Ds)); Ws_all.append(np.array(Ws))
    Ds = np.concatenate(Ds_all); Ws = np.concatenate(Ws_all)

    p_ks  = (1.0 + np.sum(Ds >= D_obs)) / (B + 1.0)
    p_cvm = (1.0 + np.sum(Ws >= W_obs)) / (B + 1.0)
    return (a_hat, b_hat, D_obs, W_obs, p_ks, p_cvm)

# ---------- GoF with selectable mode ----------
def gof_with_mode(x, fit_fun, mode="mc", B=100, seed=1234, n_jobs=N_JOBS):
    x = np.asarray(x, float); x = x[x > θ]
    if x.size == 0:
        return {"Estimator":"", "alpha":np.nan, "beta":np.nan,
                "KS_p":np.nan, "KS_D":np.nan, "CvM_p":np.nan, "CvM_W":np.nan,
                "_det":1.0/(B+1.0)}

    if mode == "mc_fixed":
        a_hat, b_hat, D_obs, W_obs, p_ks, p_cvm = mc_pvals_fixedparams(
            x, fit_fun, θ, B=B, seed=seed, n_jobs=n_jobs, nquad_obs=NQUAD_OBS, batch=MC_BATCH
        )
        return {"Estimator":"", "alpha":a_hat, "beta":b_hat,
                "KS_p":p_ks, "KS_D":D_obs, "CvM_p":p_cvm, "CvM_W":W_obs,
                "_det":1.0/(B+1.0)}

    if mode == "mc":
        a_hat, b_hat, D_obs, W_obs, p_ks, p_cvm = mc_pvals_parallel(
            x, fit_fun, θ, B=B, seed=seed, n_jobs=n_jobs,
            nquad_obs=NQUAD_OBS, nquad_mc=NQUAD_MC, grid_obs=GRID_OBS, grid_mc=GRID_MC
        )
        return {"Estimator":"", "alpha":a_hat, "beta":b_hat,
                "KS_p":p_ks, "KS_D":D_obs, "CvM_p":p_cvm, "CvM_W":W_obs,
                "_det":1.0/(B+1.0)}

    # asymptotic (not used here)
    a_hat, b_hat = fit_fun(x)
    cdf_hat = lambda z: etll_cdf(z, a_hat, b_hat, θ)
    D_obs = ks_statistic(x, cdf_hat)
    W_obs = cvm_statistic(x, cdf_hat)
    p_ks  = float(kstwobign.sf(np.sqrt(x.size) * D_obs))
    u = etll_cdf(np.sort(x), a_hat, b_hat, θ)
    p_cvm = float(cramervonmises(u, uniform.cdf).pvalue)
    return {"Estimator":"", "alpha":a_hat, "beta":b_hat,
            "KS_p":p_ks, "KS_D":D_obs, "CvM_p":p_cvm, "CvM_W":W_obs,
            "_det":0.0}



In [ ]:
# ==============================================================================
# RISK MEASURES FOR ETELL DISTRIBUTION (with ∞-CTE guard )
# ==============================================================================

# --------------------------------------------------------------------------
# VaR and CTE Calculation
# --------------------------------------------------------------------------

class ETELLRiskMeasures:
    def __init__(self, theta=500.0, n_quad=400):
        self.theta = theta
        self.n_quad = n_quad
        # Precompute Gauss-Legendre nodes and weights for [0,1]
        nodes, weights = leggauss(n_quad)
        self.u = 0.5 * (nodes + 1.0)   # map [-1,1] -> [0,1]
        self.w = 0.5 * weights         # integrates functions on [0,1] (weights sum to 1)
        self.ln2 = np.log(2.0)

    def etll_quantile(self, u, alpha, beta):
        u = np.clip(np.asarray(u, float), 1e-12, 1-1e-12)
        if abs(beta) < 1e-10:
            t = np.expm1((1.0 - u) * self.ln2)
        else:
            two_b = np.exp(beta * self.ln2)
            base  = np.maximum(two_b - (two_b - 1.0) * u, 1e-300)
            t     = np.expm1(np.log(base) / beta)
        return self.theta * np.power(np.maximum(t, 1e-300), -1.0 / alpha)

    def var(self, alpha, beta, confidence_level=0.99):
        return self.etll_quantile(confidence_level, alpha, beta)

    def cte(self, alpha, beta, confidence_level=0.99):
        # ES/CTE diverges for α ≤ 1 (heavy tail with infinite mean)
        if alpha <= 1.0:
            return float('inf')
        p = float(confidence_level)          # e.g., 0.98, 0.99, 0.995
        s = p + (1.0 - p) * self.u           # s ∈ [p,1]
        q_vals = self.etll_quantile(s, alpha, beta)
        return float(np.sum(self.w * q_vals))

    def compute_risk_measures(self, alpha, beta, confidence_levels=[0.98, 0.99, 0.995]):
        results = {}
        for cl in confidence_levels:
            tail_prob = 1.0 - cl
            var_val = self.var(alpha, beta, cl)
            cte_val = self.cte(alpha, beta, cl)
            results[cl] = {
                'confidence_level': cl,
                'tail_probability': tail_prob,
                'VaR': var_val,
                'CTE': cte_val
            }
        return results


# --------------------------------------------------------------------------
# Empirical Risk Measures
# --------------------------------------------------------------------------

def empirical_var(x, confidence_level=0.99):
    x = np.sort(np.asarray(x, float))
    return float(np.quantile(x, confidence_level, method='linear'))

def empirical_cte(x, confidence_level=0.99):
    x = np.asarray(x, float)
    var_emp = empirical_var(x, confidence_level)
    exceedances = x[x > var_emp]   # strict exceedances
    if exceedances.size == 0:
        return np.nan
    return float(np.mean(exceedances))


# --------------------------------------------------------------------------
# Risk Measures Table Generator
# --------------------------------------------------------------------------

def compute_risk_table(data_dict, estimators, confidence_levels=[0.98, 0.99, 0.995],
                       theta=500.0, n_quad=400):
    rm = ETELLRiskMeasures(theta=theta, n_quad=n_quad)
    results = {}

    for data_name, x in data_dict.items():
        x = np.asarray(x, float)
        x = x[x > theta]
        if x.size == 0:
            continue

        results[data_name] = {}

        # Empirical measures
        emp_results = {}
        for cl in confidence_levels:
            emp_results[cl] = {
                'VaR': empirical_var(x, cl),
                'CTE': empirical_cte(x, cl)
            }
        results[data_name]['Empirical'] = emp_results

        # Fitted model measures
        for est_name, fit_fun in estimators:
            try:
                alpha_hat, beta_hat = fit_fun(x)
                if not (np.isfinite(alpha_hat) and np.isfinite(beta_hat)):
                    results[data_name][est_name] = None
                    continue
                model_results = rm.compute_risk_measures(alpha_hat, beta_hat, confidence_levels)
                model_results['alpha'] = alpha_hat
                model_results['beta'] = beta_hat
                results[data_name][est_name] = model_results
            except Exception as e:
                print(f"Warning: {est_name} failed on {data_name}: {e}")
                results[data_name][est_name] = None

    return results


# ---------- helpers for pretty printing ----------
_INF_THRESHOLD = 1e15  # anything larger is printed as ∞ to avoid huge numbers

def _fmt(v):
    if v is None or not np.isfinite(v) or v > _INF_THRESHOLD:
        return "∞"
    return f"{v:,.2f}"

def _fmtdiff(a, b):
    # If either side is non-finite/huge, show dash
    bad = lambda z: (z is None) or (not np.isfinite(z)) or (z > _INF_THRESHOLD)
    if bad(a) or bad(b):
        return "—"
    return f"{(a - b):+,.2f}"


def print_risk_table(results, confidence_levels=[0.98, 0.99, 0.995]):
    for data_name, data_results in results.items():
        print("\n" + "="*100)
        print(f"Risk Measures: {data_name}")
        print("="*100)

        estimators = [k for k in data_results.keys() if k != 'Empirical']

        for cl in confidence_levels:
            tail_prob = 1.0 - cl
            print(f"\n{'Confidence Level':<20} {cl*100:.1f}% (Tail Probability = {tail_prob*100:.2f}%)")
            print("-"*100)

            # Header
            print(f"{'Method':<20} {'α̂':>8} {'β̂':>8} {'VaR':>15} {'CTE':>15} {'VaR Diff':>15} {'CTE Diff':>15}")
            print("-"*100)

            # Empirical row
            emp = data_results['Empirical'][cl]
            print(f"{'Empirical':<20} {'-':>8} {'-':>8} {_fmt(emp['VaR']):>15} {_fmt(emp['CTE']):>15} {'-':>15} {'-':>15}")

            # Model rows
            for est_name in estimators:
                est_results = data_results[est_name]
                if est_results is None or cl not in est_results:
                    print(f"{est_name:<20} {'FAILED':>8} {'FAILED':>8} {'-':>15} {'-':>15} {'-':>15} {'-':>15}")
                    continue

                alpha_hat = est_results['alpha']
                beta_hat  = est_results['beta']
                var_theo  = est_results[cl]['VaR']
                cte_theo  = est_results[cl]['CTE']

                print(f"{est_name:<20} {alpha_hat:>8.2f} {beta_hat:>8.2f} "
                      f"{_fmt(var_theo):>15} {_fmt(cte_theo):>15} "
                      f"{_fmtdiff(var_theo, emp['VaR']):>15} {_fmtdiff(cte_theo, emp['CTE']):>15}")


def print_comparison_table(results, confidence_level=0.99):
    datasets = list(results.keys())
    first_data = results[datasets[0]]
    estimators = [k for k in first_data.keys() if k != 'Empirical']

    print("\n" + "="*140)
    print(f"Risk Measures Comparison at {confidence_level*100:.1f}% Confidence Level")
    print("="*140)

    # Header
    header = f"{'Method':<20}"
    for data_name in datasets:
        header += f" | {data_name:^28}"
    print(header)

    subheader = f"{'':<20}"
    for _ in datasets:
        subheader += f" | {'VaR':>13} {'CTE':>13}"
    print(subheader)
    print("-"*140)

    # Empirical row
    row = f"{'Empirical':<20}"
    for data_name in datasets:
        emp = results[data_name]['Empirical'][confidence_level]
        row += f" | {_fmt(emp['VaR']):>13} {_fmt(emp['CTE']):>13}"
    print(row)

    print("-"*140)

    # Model rows
    for est_name in ['MLE'] + [e for e in estimators if e != 'MLE']:
        row = f"{est_name:<20}"
        for data_name in datasets:
            est_results = results[data_name].get(est_name)
            if est_results is None or confidence_level not in est_results:
                row += f" | {'FAILED':>13} {'FAILED':>13}"
            else:
                var_theo = est_results[confidence_level]['VaR']
                cte_theo = est_results[confidence_level]['CTE']
                row += f" | {_fmt(var_theo):>13} {_fmt(cte_theo):>13}"
        print(row)

    print("="*140)


# --------------------------------------------------------------------------
# Run
# --------------------------------------------------------------------------

if __name__ == "__main__":
    data_dict = {
        'Sampled (n=50)': x_sampled,
        'Modified Sampled': x_mod_sampled,
        'Original (n=9020)': x_original,
        'Modified Original': x_mod_original,
    }

    estimators = [
        ("MLE", lambda z: fit_mle_etll(z, θ)),
        ("J(1.25,9.00)", make_safe_L_factory(1.25, 9.00)),
        ("J(1.35,3.50)", make_safe_L_factory(1.35, 3.50)),
        ("J(1.30,1.60)", make_safe_L_factory(1.30, 1.60)),
        ("J(1.15,4.00)", make_safe_L_factory(1.15, 4.00)),
    ]

    print("Computing risk measures...")
    results = compute_risk_table(
        data_dict=data_dict,
        estimators=estimators,
        confidence_levels=[0.98, 0.99, 0.995],
        theta=500.0,
        n_quad=400
    )

    print_risk_table(results, confidence_levels=[0.98, 0.99, 0.995])
    print_comparison_table(results, confidence_level=0.98)
    print_comparison_table(results, confidence_level=0.99)
    print_comparison_table(results, confidence_level=0.995)


Computing risk measures...

Risk Measures: Sampled (n=50)

Confidence Level     98.0% (Tail Probability = 2.00%)
----------------------------------------------------------------------------------------------------
Method                     α̂       β̂             VaR             CTE        VaR Diff        CTE Diff
----------------------------------------------------------------------------------------------------
Empirical                   -        -        4,381.60        4,607.00               -               -
MLE                      1.77    -1.14        6,905.49       15,934.55       +2,523.89      +11,327.55
J(1.25,9.00)             1.65    -0.96        8,082.37       20,571.90       +3,700.77      +15,964.90
J(1.35,3.50)             1.47    -0.28        9,848.23       30,713.72       +5,466.63      +26,106.72
J(1.30,1.60)             1.60    -0.71        8,318.60       22,072.34       +3,937.00      +17,465.34
J(1.15,4.00)             1.65    -0.84        7,857.76       19,935